In [12]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
!pip install nltk
!pip install stop_words
!pip install gensim
!pip install pyLDAvis

In [4]:
#!/usr/bin/envpython3
# coding: utf-8

### Latent Dirichlet Allocation (LDA)

In [5]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem import RSLPStemmer
import unicodedata
from gensim import corpora, models
import gensim
import pyLDAvis.gensim

- Tokenização: separar palavras ou sentenças em unidades. Marca cada palavra como um token no texto.
- Stemmer: reduzir uma palavra ao seu radical

In [6]:
tokenizer = RegexpTokenizer(r'\w+')

# create Portuguese stop words list
pt_stop = get_stop_words('pt')

# Create p_stemmer of class PorterStemmer
p_stemmer = RSLPStemmer()

#### - Docs

In [7]:
# create sample documents
doc_a = "O brócolis é bom para comer. Meu irmão gosta de comer bons brócolis, mas não minha mãe."
doc_b = "Minha mãe passa muito tempo dirigindo para levar meu irmão para praticar futebol."
doc_c = "Alguns especialistas em saúde sugerem que a direção pode aumentar a tensão e a pressão sanguínea."
doc_d = "Costumo sentir pressão para me sair bem na escola, mas minha mãe parece nunca cobrar meu irmão para fazer melhor."
doc_e = "Profissionais da saúde dizem que brócolis faz bem a sua saúde."


In [8]:
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

#### - Pré Processamento

In [9]:
# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:

    # clean and tokenize document string
    raw = i.lower()
    #data = text
    raw = unicodedata.normalize('NFKD', raw).encode('ASCII', 'ignore')
    raw = (str(raw).replace('b\'','').replace('\'',''))

    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in pt_stop]

    #stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    #print(stemmed_tokens)

    # add tokens to list
    texts.append(stopped_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

#### LDA

In [14]:
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20,random_state=1)
#print(ldamodel)
print(ldamodel.print_topics(num_topics=2, num_words=4))

[(0, '0.066*"saude" + 0.065*"bem" + 0.039*"pressao" + 0.039*"irmao"'), (1, '0.054*"brocolis" + 0.054*"mae" + 0.054*"irmao" + 0.054*"comer"')]


#### Interpretação

- Saliency: uma medida de quanto o termo lhe diz sobre o tópico.

- Relevance: uma média ponderada da probabilidade da palavra dada ao tópico e a palavra normalizada pela probabilidade do tópico.

- O tamanho da bolha indica a importância dos tópicos em relação aos dados.

- Primeiro, temos os termos mais relevantes, significa que os termos nos dizem principalmente sobre o que está acontecendo em relação aos tópicos. É possível também observar tópicos individuais

In [15]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)